In [ ]:
#IMPORTS
import random
import csv
import heapq
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os
from glob import glob
import cv2
import numpy as np
import random
import math

In [ ]:
# Extract 3rd Problem Image zip
os.mkdir("Dataset")
file_name = "Medical_MNIST.zip"
with ZipFile(file_name, 'r') as zip:
    os.chdir("Dataset")
    zip.extractall()
os.chdir("../")

In [ ]:
# Reads Images for 3rd Problem
print(os.getcwd())
pp=[x[0] for x in os.walk("Dataset")]
data=[]
print(list(pp))
os.chdir("Dataset")
for i in range(1,len(pp)):
  os.chdir("../")
  os.chdir(pp[i])
  image_list=glob("*.jpeg")
  for mi in image_list:
    image=cv2.imread(mi,cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image,(32,32),interpolation = cv2.INTER_AREA)
    d= image.flatten()
    data.append(np.append(d,i)) 
  os.chdir("../")
os.chdir("../")
print(len(data))
data1 =data

In [ ]:
# for random sampellling of data
data = np.array(data)
data = np.random.permutation(data)

In [ ]:
# accounting for all possible outcomes
possible_outputs = [1,2,3,4,5,6]

In [ ]:
# Runs PCA to reduce Dimensionality of the Data
def PCA(data_X,k):
  X_mean=np.mean(X,axis=0)
  X_cent=data_X-X_mean
  cov_matrix=np.cov(X_cent,rowvar=False)
  eigen_values,eigen_vec=np.linalg.eigh(cov_matrix)
  index=np.argsort(eigen_values)
  index=np.flip(index)
  best_eigen=eigen_values[index]
  best_eigen_vec=eigen_vec[:,index]
  selected_eigen_vec=best_eigen_vec[:,:k]
  data_X_reduced=np.dot(selected_eigen_vec.transpose(),X_cent.transpose()).transpose()
  return data_X_reduced,best_eigen
X = (data.T[:-1]).T
y = (data.T[-1]).T
X,ev = PCA(X,5)
print(ev)

In [ ]:
#Normalizes Data
X_normed = X / np.abs(X).max(axis=0)
X = X_normed*100

In [ ]:
#Divides Data into train and test sets and other processing into dictionaries and sets
n = len(X)
data_tr = X[:(80*n)//100]
data_tt = X[(80*n)//100:]
label_tr = y[:(80*n)//100]
label_tt = y[(80*n)//100:]
label_tt = np.reshape(label_tt,(label_tt.shape[0],1))
label_tr = np.reshape(label_tr,(label_tr.shape[0],1))
data_ttt = np.append(data_tt.T,label_tt.T,axis=0).T
data_trr = np.append(data_tr.T,label_tr.T,axis=0).T
wo_data_tr_hai = dict()
wo_data_tt_hai = dict()
for i in possible_outputs:
    wo_data_tr_hai[i] = []
    wo_data_tt_hai[i] = []
for i in range(len(label_tt)):
    wo_data_tt_hai[label_tt[i][0]].append(data_tt[i])
for i in range(len(label_tr)):
    wo_data_tr_hai[label_tr[i][0]].append(data_tr[i])
dicto = dict()
for i in wo_data_tr_hai.items():
    dicto[i[0]] = len(i[1])/label_tr.shape[0]

In [ ]:
#KNN code Block 
def distance(a,b):
  return (int(a)-int(b))*(int(a)-int(b))

# predicts label for a point
def knn(data,point,k): 
  n = data.shape[0]
  o = len(point)
  heap = []
  for i in range(n):
    dist = 0
    for j in range(o):
      dist += distance(data[i][j],point[j])
    heap.append((dist,data[i][o]))
  heapq.heapify(heap)
  freq = dict()
  for i in range(k):
    temp = heapq.heappop(heap)[1]
    freq[temp]=freq.get(temp,0)+1
  max = 0
  result = 0
  for i in freq.items():
    if max<i[1]:
      max = i[1]
      result = i[0]
  return int(result)

# calls predict on test set and outputs various result parameters Confusion Matrix,Per class Accuraccy prescion F1 Score and Macro F1 Score
def knn_caller(data,test,k):
  n = test.shape[0]
  o = data.shape[1]
  macroF1 = 0
  dp = np.zeros((6,6))
  for i in range(n):
    lis = []
    for j in range(o-1):
      lis.append(test[i][j])
    temp = knn(data,lis,k)
    dp[temp-1][int(test[i][o-1]-1)]+=1
  properties = outputs(dp)
  for i in range(6):
    macroF1 += properties[i][2]/6
  return dp,properties,macroF1

In [ ]:
# Alternate Caller for KNN used for getting Confusion matrix over trining data ( nothing model related just result of bad coding practice :P )
def knn_train(wo_data_tr_hai,wo_data_tt_hai):
    lis = []
    y = []
    lis2 = []
    y2 = []
    global weights
    for i in wo_data_tr_hai.items():
        for j in i[1]:
            lis.append(j)
            y.append(i[0])
    for i in wo_data_tt_hai.items():
        for j in i[1]:
            lis2.append(j)
            y2.append(i[0])
    lis = np.array(lis)
    lis2 = np.array(lis2)
    y = np.reshape(np.array(y),(1,len(y)))
    y2 = np.reshape(np.array(y2),(1,len(y2)))
    return knn_caller(np.append(lis.T,y,axis=0).T,np.append(lis2.T,y2,axis=0).T,2)

In [ ]:
# for running bias - variance over KNN calling knn for multiple K's 
results = []
for i in range(1,10):
    results.append(knn_caller( np.append(data_tt,label_tt,axis = 1),np.append(data_tr,label_tr,axis = 1),i))

In [ ]:
# for plotting results 
graph = []
for i in range(9):
    graph.append(results[i][2])
plt.plot(np.arange(1,50),np.array(graph))

In [ ]:
#PARZEN WINDOW code

# checks if point2 lies in window around point
def valid(point,point2,parameter):
  n = len(parameter)
  for i in range(n):
    if abs(int(point2[i]) - int(point[i]))/parameter[i] > 0.5:
      return 0
  return 1

# Returns label for point given train data(Data) and window size (Parameter)
def parzen(data,point,parameter): 
  n = len(data)
  o = len(parameter)
  freq = dict()
  for i in range(n):
    freq[data[i][o]] = freq.get(data[i][o],0) + valid(point,data[i],parameter)
  max = 0
  result = 0
  for i in freq.items():
    if max<i[1]:
      max = i[1]
      result = i[0]
  return result

#Evaluates over entire test set and returns the result parameters
def parzen_caller(data,test,parameter):
  n = len(test)
  o = len(data[0])
  right = 0
  macroF1 = 0
  dp = np.zeros((6,6))
  for i in range(n):
    temp = parzen(data,test[i],parameter)
    dp[int(temp-1)][int(test[i][o-1]-1)]+=1
  properties = outputs(dp)
  for i in range(6):
    macroF1 += properties[i][2]/6
  return dp,properties,macroF1

In [ ]:
# Alternate Caller for Parzen used for getting Confusion matrix over trining data ( nothing model related just result of bad coding practice yet again :P )
def parzen_train(wo_data_tr_hai,wo_data_tt_hai):
    lis = []
    y = []
    lis2 = []
    y2 = []
    global weights
    for i in wo_data_tr_hai.items():
        for j in i[1]:
            lis.append(j)
            y.append(i[0])
    for i in wo_data_tt_hai.items():
        for j in i[1]:
            lis2.append(j)
            y2.append(i[0])
    lis = np.array(lis)
    lis2 = np.array(lis2)
    y = np.reshape(np.array(y),(1,len(y)))
    y2 = np.reshape(np.array(y2),(1,len(y2)))
    return parzen_caller(np.append(lis.T,y,axis=0).T,np.append(lis2.T,y2,axis=0).T,[25]*5)

In [ ]:
def sigmoid(X):
  return 1/(1+np.exp(-1*X))

In [ ]:
# logestic regression with all regularizer cross entropy

# some data preproceesing (adding 1 toa acount for w0)
def data_magic(X):
  nupea = []
  for i in range(len(X)):
    nupea.append( np.append(X[i],[1]) )
  return np.array(nupea)

# trains a logetistic regression for one label (one vs all used)
def loget_elastic_net(X,y,alpha,max_iter,lambda1,lambda2):
  X = data_magic(X)
  m=X.shape[0]
  n=X.shape[1]
  w = np.zeros(X.shape[1])
  y = np.reshape(y,(len(y),1))
  for i in range(max_iter):
    sgd = np.random.randint(m,size=(64))
    Xt = np.take(X,sgd,axis=0)
    yt = np.take(y,sgd)
    grad=Xt.T@(sigmoid(Xt@w)-yt)/64
    for j in range(n-1):
       if w[j]>0:
         grad[j]+=(lambda1+2*lambda2*w[j])/m
       else:
         grad[j]+=(-lambda1+2*lambda2*w[j])/m
    w=w-alpha*grad 
  return w

# trains a logetistic regression for all labels
def model_loget(X,y,alpha,max_iter,lambda1,lambda2,k=6):
  k = 6
  y1=np.zeros(len(y))
  weights=[]
  for i in range(1,7):
    y1=np.where(y==i,1,0)
    a =loget_elastic_net(X,y1,alpha,max_iter,lambda1,lambda2)
    weights.append(a)
  return weights

In [ ]:
# predicts label for a new input
def predict_loget(x):
  probab = []
  for i in weights:
    probab.append(sigmoid((np.append(x,1))@i))
  return (np.argsort(np.array(probab))[::-1])[0] + 1
    
#evaluates entire train set and returns relevant results for logisitic regression
def eval1(wo_data_tt_hai):
  dp = np.zeros((6,6))
  for i in wo_data_tt_hai.items():
    for j in i[1]:
      pre = int(predict_loget(j))
      dp[pre-1][i[0]-1]+=1
  macroF1 = 0
  properties = outputs(dp)
  for i in range(6):
    macroF1 += properties[i][2]/6
  return dp,properties,macroF1

In [ ]:
# alternate loget caller for convience 
def loget_train(wo_data_tr_hai):
    lis = []
    y = []
    global weights
    for i in wo_data_tr_hai.items():
        for j in i[1]:
            lis.append(j)
            y.append(i[0])
    print(y)
    weights = model_loget(lis,np.array(y),0.005,10000,2,0)

In [ ]:
# given confusion matrix evaluates per class acuuracy presion recall etc
def confusion(dp,k):
  tp = 0 
  tn = 0
  fp = 0
  fn = 0
  for i in range(6):
      for j in range(6):
          if i==j==k:
              tp+=dp[i][j]
          elif i==k and j!=k:
              fp+=dp[i][j]
          elif j==k and i!=k:
              fn+=dp[i][j]
          else:
              tn+=dp[i][j]
  prescion = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = 2/(1/recall+1/prescion)
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  return accuracy,prescion,f1,recall,tp,fp,tn,fn
def outputs(dp):
    properties = []
    for i in range(6):
        properties.append(confusion(dp,i))
    return np.array(properties)

In [ ]:
#fits gaussian model for a given data
def fit_gaussian(data_tr):
  data_tr = np.array(data_tr)
  return (np.reshape(np.mean(data_tr,axis=0),(20,1)),np.cov(data_tr.T,rowvar=1))

# trains gaussian for all labels
dict1 = {}
def train(wo_data_tr_hai):
  global dict1 , dicto
  dict1 = {}
  for i in wo_data_tr_hai.items():
    dicto[i[0]] = len(i[1])/label_tr.shape[0]
  for i in wo_data_tr_hai.items():
    dict1[i[0]] = fit_gaussian(i[1])

In [ ]:
e = math.exp(1)
# multivariate logarithm of a gaussian function
def gaussian(x,sigma,u,n):
  x = np.reshape(x,(x.shape[0],1))
  const =  1/math.sqrt( ( abs(np.linalg.det(sigma)) ) ) 
  other_term = ((np.transpose(x-u))@(np.linalg.pinv(sigma)))@(x-u)
  return math.log(const,e)  + -0.5*other_term

# predicts label for a input
def predict(x):
  max = -1*math.inf
  label = "default label"
  for i in dict1.items():
    temp = gaussian(x,i[1][1],i[1][0],len(x))
    if max < temp + math.log(dicto[i[0]],e):
      label = i[0]
      max = temp + math.log(dicto[i[0]],e)
  return label
  
# evaluates test data and returns relevant information
def check(wo_data_tt_hai):
  right = 0
  l = 0
  dp = np.zeros((6,6))
  for i in wo_data_tt_hai.items():
    for j in i[1]:
      pre = int(predict(j))
      dp[pre-1][i[0]-1]+=1
      if i[0] == pre:
        right+=1
  macroF1 = 0
  properties = outputs(dp)
  for i in range(6):
    macroF1 += properties[i][2]/6
  return dp,properties,macroF1

In [ ]:
#splits the train data into size of 5 for 5 cross analysis
lis = []
for j in wo_data_tr_hai.items():
    lis2 = []
    ss = np.array(j[1])
    n = ss.shape[0]
    for i in range(5):
        lis2.append(ss[(i*n)//5:((i+1)*n)//5])
    lis.append(lis2)

In [ ]:
# runs 5 corss validation over train data and returns Confusion matrix
Confusion_in_this_question = np.zeros((6,6))
for i in range(5):
    test = dict()
    train1 = dict()
    for j in range(5):
        for k in range(6):
            if j!=i:
                if k+1 in train1.keys():
                    train1[k+1] = np.append(train1[k+1],lis[k][j],axis=0)
                else:
                    train1[k+1] = lis[k][j]
            else:
                test[k+1] = lis[k][i]
    #training function
    #test function
    Confusion_in_this_question += knn_train(train1,test)[0]/5
print(Confusion_in_this_question)

In [ ]:
# fits unimodal gaussian over entire data
def sasta_fit_gaussian(data_tr):
  data_tr = np.array(data_tr)
  return (np.mean(data_tr),np.var(data_tr))
dict1 = {}

In [ ]:
# Just due to bad naming practices :(
possible_outcomes = possible_outputs 

In [ ]:
# univariate gaussian is sasta gaussian sorry :\
# Naive Bayes Caller logic 
e = math.exp(1)
#univariate gaussian evaluate 
def sasta_gaussian(x,sigma,u):
  return (1/math.sqrt(math.pi*2*sigma))*math.exp(-0.5*((x-u)*(x-u)/sigma)) 

# evaluates models for all naive bayes
def naive_bayes(data_tr):
  n = 20
  models = np.zeros((20,6,2))
  for i in range(n):
    for j in possible_outcomes:
      models[i][j-1][0],models[i][j-1][1] = sasta_fit_gaussian( (np.array(data_tr[j]).T[i]) ) 
  return models

models = naive_bayes(wo_data_tr_hai)

# Evaluates test data and returns releveant results 
def eval(data_tt):
  l = 0
  dp = np.zeros((6,6))
  for i in wo_data_tt_hai.items():
    for j in i[1]:
      pre = int(predict_n(j))
      dp[pre-1][i[0]-1]+=1
  macroF1 = 0
  properties = outputs(dp)
  for i in range(6):
    macroF1 += properties[i][2]/6
  return dp,properties,macroF1

# predicts labal for a given point
def predict_n(x):
  max = -1*math.inf
  label = 'default label'
  for i in range(len(possible_outcomes)):
    temp = dicto[i+1]
    for j in range(20):
      temp*=sasta_gaussian(x[j],models[j][i][1],models[j][i][0])
    if temp > max:
      max = temp
      label = possible_outcomes[i]
  return label 

print(eval(wo_data_tt_hai))

In [ ]:
# EM algoritm for GMM returns mu ,wights and sigma
e =math.exp(1)
def GM(data,iteration,k):
  data = np.array(data)
  dimension = len(data[0])
  n = len(data)
  pk = np.array( [1/k]*k )
  mu = np.array( np.random.uniform(low=0.01,high=0.02, size=(k,dimension,1)) )
  sigma = np.array( np.random.uniform(low=0.001,high=1000, size=(k,dimension,dimension)) )
  for iter in range(iteration):
    tmu = np.zeros((k,dimension,1))
    tsigma = np.zeros((k,dimension,dimension))
    tpk = np.zeros((k,1))
    global_sum = 0 

    dp = np.zeros((n,k))
    #E - Step
    for i in range(n):
      normal = 0
      for j in range(k):
        oo = np.reshape(data[i],(data[i].shape[0],1))
        dp[i][j] = pk[j]*math.exp(gaussian(oo,sigma[j],mu[j],dimension))
        normal += dp[i][j]
      dp[i]/=normal
      for j in range(k):
        global_sum+=dp[i][j]
        tpk[j]+=dp[i][j]
      
    # M - step
    for i in range(n):
      for j in range(k):
        oo = np.reshape(data[i],(data[i].shape[0],1))
        factor =  (dp[i][j]/tpk[j])
        tmu[j] += factor*oo
        oj = (oo-mu[j])@(np.transpose(oo-mu[j]))
        tsigma[j] += factor*oj

    for i in range(k):
      tpk[i]/=global_sum

    #Exchange 
    pk = tpk.copy()
    sigma = tsigma.copy()
    mu = tmu.copy()
  return [pk,mu,sigma]
    

In [ ]:
# caller to train GMM
pp = dict()
def gmmtrain(wo_data_tr_hai,k):
    global pp
    pp = dict()
    for i in wo_data_tr_hai.items():
        pp[i[0]] = GM(i[1],50,k)  

In [ ]:
#returns label for a point x 
def predict_gmm(x,k):
  probab = []
  for i in pp.items():
    sum = 0
    for j in range(k):
      sum += i[1][0][j]*math.exp(gaussian(x,i[1][2][j],i[1][1][j],len(x)))
    probab.append(dicto[i[0]]*sum)
  probab = np.array(probab)
  max = 0
  label = -1
  for i in range(6):
      if probab[i] > max:
          max = probab[i]
          label = i+1 
  return label

# Evaluates test set and returns results like confusion matrix macro F1 etc
def check_gmm(wo_data_tt_hai,k):
  right = 0
  l = 0
  dp = np.zeros((6,6))
  for i in wo_data_tt_hai.items():
    for j in i[1]:
      pre = int(predict_gmm(j,k))
      dp[pre-1][i[0]-1]+=1
      if i[0] == pre:
        right+=1
  macroF1 = 0
  properties = outputs(dp)
  for i in range(6):
    macroF1 += properties[i][2]/6
  return dp,properties,macroF1

In [ ]:
#return train error for GMM model
def train_error_gmm(k):
  right = 0
  l = 0
  dp = np.zeros((6,6))
  for i in wo_data_tr_hai.items():
    for j in i[1]:
      pre = int(predict_gmm(j,k))
      dp[pre-1][i[0]-1]+=1
      if i[0] == pre:
        right+=1
  macroF1 = 0
  properties = outputs(dp)
  for i in range(6):
    macroF1 += properties[i][2]/6
  return dp,properties,macroF1